In [15]:
import json
import pandas as pd
import os
import numpy as np

### Read Dataset from json

In [43]:
def read_json(file_path):
    data = []
    for file in os.listdir(file_path):
        if file.endswith(".json"):
            with open(os.path.join(file_path, file), 'r') as f:
                json_data = json.load(f)
                data.append(json_data)
    return data

file_path = 'data/json/'

data = read_json(file_path)


In [75]:

df = pd.DataFrame(data)
df.to_csv('data/data.csv', index=False)

In [32]:
df.head()

,url,date,title,content
0,https://news.detik.com/berita/d-4047915/soal-t...,2018-05-31 23:13:00,"Soal THR untuk Anggota DPR, NasDem: Itu Normal",Jakarta -\nPimpinan dan anggota DPR tahun ini ...
1,https://bola.kompas.com/read/2018/05/31/232609...,2018-05-31 23:26:00,"Sempat Unggul, Timnas U-23 Indonesia Kalah dar...",KOMPAS.com - Timnas U-23 Indonesia harus menga...
2,https://nasional.kompas.com/read/2018/05/31/23...,2018-05-31 23:32:00,Saat Oesman Sapta Candai Capres dan Cawapres d...,"JAKARTA, KOMPAS.com - Ketua Dewan Pimpinan Dae..."
3,https://international.sindonews.com/read/13106...,2018-05-31 23:47:00,Assad: Israel Panik Kehilangan Teroris Tersaya...,DAMASKUS - Presiden Bashar al-Assad menyebut I...
4,https://finance.detik.com/detiktv/d-4047895/du...,2018-05-31 23:07:00,"Dua Minggu Jelang Lebaran, Pegadaian Dibanjiri...","Dua minggu jelang Lebaran, transaksi di PT Peg..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      61 non-null     object
 1   date     61 non-null     object
 2   title    61 non-null     object
 3   content  61 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


### Text Preprocessing

In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Case folding

In [76]:
import re

def case_folding(text):
    """
    param text: string
    return: string

    Lowercase the text, remove url, remove number, remove punctuation, remove whitespace, remove trailing & leading whitespace
    """
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) # remove url
    text = re.sub(r'[-+]?[0-9]+', '', text) # remove number
    text = re.sub(r"[^\w]", " ", text) # remove punctuation
    text = re.sub(r"\s+", " ", text) # remove whitespace
    text = text.strip() # remove trailing & leading whitespace
    return text

In [34]:
raw_sample = df['content'].iloc[5]
case_folding = case_folding(raw_sample)

print(f'raw sample: {raw_sample}')
print(f'case folding: {case_folding}')

raw sample: Jakarta -
Menteri Perhubungan Budi Karya Sumadi setuju penumpang yang bercanda membawa bom di pesawat Lion Air JT 687 rute Pontianak-Cengkareng di- blacklist . Hal tersebut agar tidak ada lagi penumpang yang bercanda soal bom di pesawat. "Saya pikir suatu usulan yang baik ya, karena bom itu bukan candaan, bukan yang lucu. Jadi saya pikir, jika seseorang melakukan hal itu, patut untuk dilakukan blacklist ," ujar Budi di Terminal Penumpang Pelabuhan Tanjung Priok, Jakarta Utara, Kamis (31/5/2018). Menurut Budi, kekisruhan di dalam pesawat akibat candaan bom tidak perlu lagi terjadi. Candaan bom bukan candaan yang lucu. "Karena kita tidak ingin ada suatu kekisruhan dalam pesawat dengan candaan yang nggak lucu, yang sebenarnya nggak penting," katanya. Budi mengimbau masyarakat agar tidak meniru candaan tersebut. Penumpang di mana pun harus tetap disiplin. "Jadi saya pikir kita harus menghindari, harus disiplin supaya candaan nggak lucu tidak disampaikan," jelasnya. Sebelumnya, 

### Slang World Normalization

In [35]:
key_norm = pd.read_csv('data/key_norm.csv')
key_norm.shape

(3720, 3)

In [36]:
key_norm.head()

,_id,singkat,hasil
0,1,abis,habis
1,2,accent,tekanan
2,3,accept,terima
3,4,accident,kecelakaan
4,5,achievement,prestasi


In [77]:
def text_normalize(text):
    """
    param text: string
    return: string

    Normalize the text by replacing the key with the value in the key_norm dataframe
    """
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
    return text


In [46]:
raw_sample = df['content'].iloc[5]
case_folding = case_folding(raw_sample)
text_normalize = text_normalize(case_folding)

print(f'raw sample: {raw_sample}')
print(f'case folding: {case_folding}')
print(f'text normalize: {text_normalize}')

raw sample: Jakarta -
Menteri Perhubungan Budi Karya Sumadi setuju penumpang yang bercanda membawa bom di pesawat Lion Air JT 687 rute Pontianak-Cengkareng di- blacklist . Hal tersebut agar tidak ada lagi penumpang yang bercanda soal bom di pesawat. "Saya pikir suatu usulan yang baik ya, karena bom itu bukan candaan, bukan yang lucu. Jadi saya pikir, jika seseorang melakukan hal itu, patut untuk dilakukan blacklist ," ujar Budi di Terminal Penumpang Pelabuhan Tanjung Priok, Jakarta Utara, Kamis (31/5/2018). Menurut Budi, kekisruhan di dalam pesawat akibat candaan bom tidak perlu lagi terjadi. Candaan bom bukan candaan yang lucu. "Karena kita tidak ingin ada suatu kekisruhan dalam pesawat dengan candaan yang nggak lucu, yang sebenarnya nggak penting," katanya. Budi mengimbau masyarakat agar tidak meniru candaan tersebut. Penumpang di mana pun harus tetap disiplin. "Jadi saya pikir kita harus menghindari, harus disiplin supaya candaan nggak lucu tidak disampaikan," jelasnya. Sebelumnya, 

### Stopword Removal

In [56]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

len(stopwords_ind)

758

In [78]:
more_stopword = ['nvl']

stopwords_ind.extend(more_stopword)

def remove_stopwords(text):
    """
    param text: string
    return: string

    Remove stopwords from the text
    """
    text = ' '.join([word for word in text.split() if word not in stopwords_ind])
    return text

In [63]:
raw_sample = df['content'].iloc[5]
case_folding = case_folding(raw_sample)
text_normalize = text_normalize(case_folding)
remove_stopwords = remove_stopwords(text_normalize)

print(f'raw sample: {raw_sample}')
print(f'case folding: {case_folding}')
print(f'text normalize: {text_normalize}')
print(f'remove stopwords: {remove_stopwords}')


raw sample: Jakarta -
Menteri Perhubungan Budi Karya Sumadi setuju penumpang yang bercanda membawa bom di pesawat Lion Air JT 687 rute Pontianak-Cengkareng di- blacklist . Hal tersebut agar tidak ada lagi penumpang yang bercanda soal bom di pesawat. "Saya pikir suatu usulan yang baik ya, karena bom itu bukan candaan, bukan yang lucu. Jadi saya pikir, jika seseorang melakukan hal itu, patut untuk dilakukan blacklist ," ujar Budi di Terminal Penumpang Pelabuhan Tanjung Priok, Jakarta Utara, Kamis (31/5/2018). Menurut Budi, kekisruhan di dalam pesawat akibat candaan bom tidak perlu lagi terjadi. Candaan bom bukan candaan yang lucu. "Karena kita tidak ingin ada suatu kekisruhan dalam pesawat dengan candaan yang nggak lucu, yang sebenarnya nggak penting," katanya. Budi mengimbau masyarakat agar tidak meniru candaan tersebut. Penumpang di mana pun harus tetap disiplin. "Jadi saya pikir kita harus menghindari, harus disiplin supaya candaan nggak lucu tidak disampaikan," jelasnya. Sebelumnya, 

### Stemming

In [79]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    """
    param text: string
    return: string

    Stemming the text
    """
    text = stemmer.stem(text)
    return text

In [74]:
raw_sample = df['content'].iloc[5]
case_folding = case_folding(raw_sample)
text_normalize = text_normalize(case_folding)
remove_stopwords = remove_stopwords(text_normalize)
stemming = stemming(remove_stopwords)

print(f'raw sample: {raw_sample}')
print(f'case folding: {case_folding}')
print(f'text normalize: {text_normalize}')
print(f'remove stopwords: {remove_stopwords}')
print(f'stemming: {stemming}')


raw sample: Jakarta -
Menteri Perhubungan Budi Karya Sumadi setuju penumpang yang bercanda membawa bom di pesawat Lion Air JT 687 rute Pontianak-Cengkareng di- blacklist . Hal tersebut agar tidak ada lagi penumpang yang bercanda soal bom di pesawat. "Saya pikir suatu usulan yang baik ya, karena bom itu bukan candaan, bukan yang lucu. Jadi saya pikir, jika seseorang melakukan hal itu, patut untuk dilakukan blacklist ," ujar Budi di Terminal Penumpang Pelabuhan Tanjung Priok, Jakarta Utara, Kamis (31/5/2018). Menurut Budi, kekisruhan di dalam pesawat akibat candaan bom tidak perlu lagi terjadi. Candaan bom bukan candaan yang lucu. "Karena kita tidak ingin ada suatu kekisruhan dalam pesawat dengan candaan yang nggak lucu, yang sebenarnya nggak penting," katanya. Budi mengimbau masyarakat agar tidak meniru candaan tersebut. Penumpang di mana pun harus tetap disiplin. "Jadi saya pikir kita harus menghindari, harus disiplin supaya candaan nggak lucu tidak disampaikan," jelasnya. Sebelumnya, 

### Text Preprocessing Pipeline

In [80]:
def text_preprocessing_pipeline(text):
    """
    param text: string
    return: string

    Apply text preprocessing pipeline
    """
    text = case_folding(text)
    text = text_normalize(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return text

In [81]:
%%time

df['content_clean'] = df['content'].apply(text_preprocessing_pipeline)

CPU times: user 3min 12s, sys: 196 ms, total: 3min 12s
Wall time: 3min 14s


In [82]:
df.to_csv('data/data_clean.csv', index=False)

In [83]:
df.head()

,url,date,title,content,content_clean
0,https://news.detik.com/berita/d-4047915/soal-t...,2018-05-31 23:13:00,"Soal THR untuk Anggota DPR, NasDem: Itu Normal",Jakarta -\nPimpinan dan anggota DPR tahun ini ...,jakarta pimpin anggota dpr terima tunjang raya...
1,https://bola.kompas.com/read/2018/05/31/232609...,2018-05-31 23:26:00,"Sempat Unggul, Timnas U-23 Indonesia Kalah dar...",KOMPAS.com - Timnas U-23 Indonesia harus menga...,kompas com tim nasional indonesia aku unggul t...
2,https://nasional.kompas.com/read/2018/05/31/23...,2018-05-31 23:32:00,Saat Oesman Sapta Candai Capres dan Cawapres d...,"JAKARTA, KOMPAS.com - Ketua Dewan Pimpinan Dae...",jakarta kompas com ketua dewan pimpin daerah d...
3,https://international.sindonews.com/read/13106...,2018-05-31 23:47:00,Assad: Israel Panik Kehilangan Teroris Tersaya...,DAMASKUS - Presiden Bashar al-Assad menyebut I...,damaskus presiden bashar al assad sebut israel...
4,https://finance.detik.com/detiktv/d-4047895/du...,2018-05-31 23:07:00,"Dua Minggu Jelang Lebaran, Pegadaian Dibanjiri...","Dua minggu jelang Lebaran, transaksi di PT Peg...",minggu jelang lebaran transaksi pt gadai perse...
